# Cleaning and Preprocessing Data for Machine Learning

In [1]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, date
from google.colab import files

In [2]:
# Read results csv file into a pandas DataFrame

results_clean_df = pd.read_csv('https://sumo-data-bucket.s3.amazonaws.com/results_clean.csv')
results_clean_df.drop(results_clean_df.columns[[0]],axis=1,inplace=True)
results_clean_df.tail()

,tournament_date,day,wrestler1_id,wrestler1_rank,wrestler1_name,wrestler1_result,wrestler1_win,finishing_move,wrestler2_id,wrestler2_rank,wrestler2_name,wrestler2_result,wrestler2_win
219985,2021-05-01,15,12130,O2e,Shodai,9-6,1,oshidashi,12055,M8w,Endo,11-4,0
219986,2021-05-01,15,11927,O2w,Terunofuji,12-3,0,tsukiotoshi,12191,O1w,Takakeisho,12-3,1
219987,2021-05-01,15,12191,O1w,Takakeisho,12-3,1,tsukiotoshi,11927,O2w,Terunofuji,12-3,0
219988,2021-05-01,16,11927,O2w,Terunofuji,(12-3),1,hatakikomi,12191,O1w,Takakeisho,(12-3),0
219989,2021-05-01,16,12191,O1w,Takakeisho,(12-3),0,hatakikomi,11927,O2w,Terunofuji,(12-3),1


In [3]:
# Read banzuke csv file into a pandas DataFrame

banzuke_clean_df = pd.read_csv('https://sumo-data-bucket.s3.amazonaws.com/banzuke_clean.csv')
banzuke_clean_df.drop(banzuke_clean_df.columns[[0]],axis=1,inplace=True)
banzuke_clean_df.tail()

,tournament_date,id,rank,wrestler,stable,birth_place,birth_date,height,weight,previous_rank,previous_wins,previous_losses
15392,2021-05-01,12351,J12w,Nishikifuji,Isegahama,Aomori,1996-07-22,183.0,143.0,J12w,7.0,8.0
15393,2021-05-01,12040,J13e,Daishoho,Oitekaze,Mongolia,1994-08-28,184.0,171.0,Ms2w,5.0,2.0
15394,2021-05-01,6596,J13w,Nishikigi,Isenoumi,Iwate,1990-08-25,184.0,169.0,J13w,7.0,8.0
15395,2021-05-01,12117,J14e,Bushozan,Fujishima,Ibaraki,1995-12-06,172.0,170.0,J14e,7.0,8.0
15396,2021-05-01,12453,J14w,Oho,Otake,Tokyo,2000-02-14,191.0,170.0,Ms2e,4.0,3.0


In [4]:
#Prep banzuke file for merge, rename id to wrestler1_id 
banzuke_clean_df.rename(columns = {'id':'wrestler1_id'}, inplace = True)
#Merge wrestler1 info into new DataFrame called sumo_df
sumo_df = pd.merge(results_clean_df, banzuke_clean_df, how="inner", on=['tournament_date','wrestler1_id'])
#Drop undesirable details for wrestler1 from banzuke_df
sumo_df.drop(['rank','wrestler','previous_rank','previous_wins','previous_losses'],axis=1, inplace = True)
#Rename columns for wrestler1 info to distinguish from wrestler2 info in future step(s)
sumo_df.rename(columns=({'stable':'wrestler_1_stable','birth_place':'wrestler_1_birth_place','birth_date':'wrestler_1_birth_date','height':'wrestler_1_height','weight':'wrestler_1_weight'}), inplace = True)
#show results
sumo_df

,tournament_date,day,wrestler1_id,wrestler1_rank,wrestler1_name,wrestler1_result,wrestler1_win,finishing_move,wrestler2_id,wrestler2_rank,wrestler2_name,wrestler2_result,wrestler2_win,wrestler_1_stable,wrestler_1_birth_place,wrestler_1_birth_date,wrestler_1_height,wrestler_1_weight
0,1983-01-01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1,Miyagino,Fukuoka,1957-08-21,175.5,115.0
1,1983-01-01,2,4140,J13w,Chikubayama,1-1 (7-8),1,oshidashi,4125,J7e,Shinko,0-2 (8-7),0,Miyagino,Fukuoka,1957-08-21,175.5,115.0
2,1983-01-01,3,4140,J13w,Chikubayama,2-1 (7-8),1,oshidashi,4318,J8w,Tengoyama,1-2 (7-8),0,Miyagino,Fukuoka,1957-08-21,175.5,115.0
3,1983-01-01,4,4140,J13w,Chikubayama,2-2 (7-8),0,hikiotoshi,4322,J5e,Sanofuji,2-2 (6-9),1,Miyagino,Fukuoka,1957-08-21,175.5,115.0
4,1983-01-01,5,4140,J13w,Chikubayama,2-3 (7-8),0,yorikiri,1302,J9e,Takamisugi,2-3 (5-10),1,Miyagino,Fukuoka,1957-08-21,175.5,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217151,2021-05-01,12,11786,M3e,Aoiyama,2-2-8 (4-3-8),1,hatakikomi,11784,M4w,Myogiryu,5-7 (6-9),0,Kasugano,Bulgaria,1986-06-19,191.0,188.0
217152,2021-05-01,13,11786,M3e,Aoiyama,3-2-8 (4-3-8),1,hatakikomi,12273,M13w,Daiamami,6-7 (7-8),0,Kasugano,Bulgaria,1986-06-19,191.0,188.0
217153,2021-05-01,14,11786,M3e,Aoiyama,4-2-8 (4-3-8),1,tsukihiza,11918,M17e,Akua,4-10 (5-10),0,Kasugano,Bulgaria,1986-06-19,191.0,188.0
217154,2021-05-01,15,11786,M3e,Aoiyama,4-3-8,0,shitatenage,11868,M10w,Terutsuyoshi,7-8,1,Kasugano,Bulgaria,1986-06-19,191.0,188.0


In [5]:
#Prep banzuke file for merge, rename wrestler1_id to wrestler2_id
banzuke_clean_df.rename(columns = {'wrestler1_id':'wrestler2_id'}, inplace = True)
#Merge wrestler2 info into new DataFrame called sumo_df
sumo_df = pd.merge(sumo_df, banzuke_clean_df, how="inner", on=['tournament_date','wrestler2_id'])
#Drop undesirable details for wrestler2 from banzuke_df
sumo_df.drop(['rank','wrestler','previous_rank','previous_wins','previous_losses'],axis=1, inplace = True)
#Rename columns for wrestler2 info to distinguish from wrestler1 info in prior step(s)
sumo_df.rename(columns=({'stable':'wrestler_2_stable','birth_place':'wrestler_2_birth_place','birth_date':'wrestler_2_birth_date','height':'wrestler_2_height','weight':'wrestler_2_weight'}), inplace = True)
sumo_2_df = sumo_df
sumo_df

,tournament_date,day,wrestler1_id,wrestler1_rank,wrestler1_name,wrestler1_result,wrestler1_win,finishing_move,wrestler2_id,wrestler2_rank,wrestler2_name,wrestler2_result,wrestler2_win,wrestler_1_stable,wrestler_1_birth_place,wrestler_1_birth_date,wrestler_1_height,wrestler_1_weight,wrestler_2_stable,wrestler_2_birth_place,wrestler_2_birth_date,wrestler_2_height,wrestler_2_weight
0,1983-01-01,2,4140,J13w,Chikubayama,1-1 (7-8),1,oshidashi,4125,J7e,Shinko,0-2 (8-7),0,Miyagino,Fukuoka,1957-08-21,175.5,115.0,Isegahama,Yamagata,1950-01-02,188.0,134.0
1,1983-01-01,8,1337,J12w,Tochitsukasa,4-4 (9-6),1,oshidashi,4125,J7e,Shinko,4-4 (8-7),0,Kasugano,Aichi,1958-04-25,178.5,130.0,Isegahama,Yamagata,1950-01-02,188.0,134.0
2,1983-01-01,11,4097,J12e,Tamakiyama,4-7 (8-7),0,uwatenage,4125,J7e,Shinko,6-5 (8-7),1,Kataonami,Fukuoka,1951-04-29,182.0,134.0,Isegahama,Yamagata,1950-01-02,188.0,134.0
3,1983-01-01,6,4319,J11w,Harunafuji,3-3 (5-10),1,yorikiri,4125,J7e,Shinko,3-3 (8-7),0,Taiho,Gunma,1959-02-03,178.3,156.0,Isegahama,Yamagata,1950-01-02,188.0,134.0
4,1983-01-01,5,4129,J11e,Hakuryuyama,2-3 (3-12),0,hikiotoshi,4125,J7e,Shinko,3-2 (8-7),1,Tokitsukaze,Hyogo,1954-07-10,178.5,113.0,Isegahama,Yamagata,1950-01-02,188.0,134.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214317,2021-05-01,12,12291,O1e,Asanoyama,7-5 (7-5-3),0,fusen,6480,S1e,Takayasu,8-4 (10-5),1,Takasago,Toyama,1994-03-01,187.0,174.0,Tagonoura,Ibaraki,1990-02-28,187.0,175.0
214318,2021-05-01,3,12210,K1e,Mitakeumi,3-0 (10-5),1,oshidashi,6642,M3w,Chiyonokuni,0-3 (0-4-11),0,Dewanoumi,Nagano,1992-12-25,179.0,172.0,Kokonoe,Mie,1990-07-10,182.0,144.0
214319,2021-05-01,1,11855,S1w,Takanosho,1-0 (5-10),1,oshidashi,6642,M3w,Chiyonokuni,0-1 (0-4-11),0,Tokiwayama,Chiba,1994-11-14,184.0,163.0,Kokonoe,Mie,1990-07-10,182.0,144.0
214320,2021-05-01,2,6480,S1e,Takayasu,2-0 (10-5),1,tsukitaoshi,6642,M3w,Chiyonokuni,0-2 (0-4-11),0,Tagonoura,Ibaraki,1990-02-28,187.0,175.0,Kokonoe,Mie,1990-07-10,182.0,144.0


In [6]:
#Clean up dtypes for future step(s)

#convert tournament_date, wrestler_1_birth_date, and wrestler_2_birth_date from object to datetime
sumo_df['wrestler_1_birth_date'] =  pd.to_datetime(sumo_df['wrestler_1_birth_date'], format='%Y-%m-%d')
sumo_df['wrestler_2_birth_date'] =  pd.to_datetime(sumo_df['wrestler_2_birth_date'], format='%Y-%m-%d')
sumo_df['tournament_date'] =  pd.to_datetime(sumo_df['tournament_date'], format='%Y-%m-%d')
#drop empty dates (if any)
sumo_df.dropna(inplace=True)
#show results
sumo_df.tail()

,tournament_date,day,wrestler1_id,wrestler1_rank,wrestler1_name,wrestler1_result,wrestler1_win,finishing_move,wrestler2_id,wrestler2_rank,wrestler2_name,wrestler2_result,wrestler2_win,wrestler_1_stable,wrestler_1_birth_place,wrestler_1_birth_date,wrestler_1_height,wrestler_1_weight,wrestler_2_stable,wrestler_2_birth_place,wrestler_2_birth_date,wrestler_2_height,wrestler_2_weight
214317,2021-05-01,12,12291,O1e,Asanoyama,7-5 (7-5-3),0,fusen,6480,S1e,Takayasu,8-4 (10-5),1,Takasago,Toyama,1994-03-01,187.0,174.0,Tagonoura,Ibaraki,1990-02-28,187.0,175.0
214318,2021-05-01,3,12210,K1e,Mitakeumi,3-0 (10-5),1,oshidashi,6642,M3w,Chiyonokuni,0-3 (0-4-11),0,Dewanoumi,Nagano,1992-12-25,179.0,172.0,Kokonoe,Mie,1990-07-10,182.0,144.0
214319,2021-05-01,1,11855,S1w,Takanosho,1-0 (5-10),1,oshidashi,6642,M3w,Chiyonokuni,0-1 (0-4-11),0,Tokiwayama,Chiba,1994-11-14,184.0,163.0,Kokonoe,Mie,1990-07-10,182.0,144.0
214320,2021-05-01,2,6480,S1e,Takayasu,2-0 (10-5),1,tsukitaoshi,6642,M3w,Chiyonokuni,0-2 (0-4-11),0,Tagonoura,Ibaraki,1990-02-28,187.0,175.0,Kokonoe,Mie,1990-07-10,182.0,144.0
214321,2021-05-01,4,11985,K1w,Daieisho,2-2 (6-9),1,fusen,6642,M3w,Chiyonokuni,0-4 (0-4-11),0,Oitekaze,Saitama,1993-11-10,182.0,161.0,Kokonoe,Mie,1990-07-10,182.0,144.0


In [7]:
#Calculate wrestler ages at each tournament_date
#Create separate DataFrame containing the dates used for calculations
age_df = sumo_df[['tournament_date','wrestler_1_birth_date','wrestler_2_birth_date']]
#Calculate wrestler1 age
age_df['wrestler_1_age'] = age_df['tournament_date'].dt.year - age_df['wrestler_1_birth_date'].dt.year
#Calculate wrestler2 age
age_df['wrestler_2_age'] = age_df['tournament_date'].dt.year - age_df['wrestler_2_birth_date'].dt.year
#Drop date columns, leaving only the ages
del age_df['wrestler_1_birth_date']
del age_df['wrestler_2_birth_date']
del age_df['tournament_date']
#Show results
age_df.tail()

,wrestler_1_age,wrestler_2_age
214317,27,31
214318,29,31
214319,27,31
214320,31,31
214321,28,31


In [8]:
#Merge ages into sumo_df
sumo_df = pd.merge(sumo_df, age_df, how="inner", left_index=True, right_index=True)
#show final DataFrame
sumo_df.tail()

,tournament_date,day,wrestler1_id,wrestler1_rank,wrestler1_name,wrestler1_result,wrestler1_win,finishing_move,wrestler2_id,wrestler2_rank,wrestler2_name,wrestler2_result,wrestler2_win,wrestler_1_stable,wrestler_1_birth_place,wrestler_1_birth_date,wrestler_1_height,wrestler_1_weight,wrestler_2_stable,wrestler_2_birth_place,wrestler_2_birth_date,wrestler_2_height,wrestler_2_weight,wrestler_1_age,wrestler_2_age
214317,2021-05-01,12,12291,O1e,Asanoyama,7-5 (7-5-3),0,fusen,6480,S1e,Takayasu,8-4 (10-5),1,Takasago,Toyama,1994-03-01,187.0,174.0,Tagonoura,Ibaraki,1990-02-28,187.0,175.0,27,31
214318,2021-05-01,3,12210,K1e,Mitakeumi,3-0 (10-5),1,oshidashi,6642,M3w,Chiyonokuni,0-3 (0-4-11),0,Dewanoumi,Nagano,1992-12-25,179.0,172.0,Kokonoe,Mie,1990-07-10,182.0,144.0,29,31
214319,2021-05-01,1,11855,S1w,Takanosho,1-0 (5-10),1,oshidashi,6642,M3w,Chiyonokuni,0-1 (0-4-11),0,Tokiwayama,Chiba,1994-11-14,184.0,163.0,Kokonoe,Mie,1990-07-10,182.0,144.0,27,31
214320,2021-05-01,2,6480,S1e,Takayasu,2-0 (10-5),1,tsukitaoshi,6642,M3w,Chiyonokuni,0-2 (0-4-11),0,Tagonoura,Ibaraki,1990-02-28,187.0,175.0,Kokonoe,Mie,1990-07-10,182.0,144.0,31,31
214321,2021-05-01,4,11985,K1w,Daieisho,2-2 (6-9),1,fusen,6642,M3w,Chiyonokuni,0-4 (0-4-11),0,Oitekaze,Saitama,1993-11-10,182.0,161.0,Kokonoe,Mie,1990-07-10,182.0,144.0,28,31


In [9]:
#Save file of complete DataFrame 
sumo_df.to_csv('sumo_data_complete.csv', index = False)
files.download('sumo_data_complete.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
#Drop pre-2004 results and save file for project use
filtered_df = sumo_df.loc[(sumo_df['tournament_date'] >= '2004-01-01')]
filtered_df.head()

,tournament_date,day,wrestler1_id,wrestler1_rank,wrestler1_name,wrestler1_result,wrestler1_win,finishing_move,wrestler2_id,wrestler2_rank,wrestler2_name,wrestler2_result,wrestler2_win,wrestler_1_stable,wrestler_1_birth_place,wrestler_1_birth_date,wrestler_1_height,wrestler_1_weight,wrestler_2_stable,wrestler_2_birth_place,wrestler_2_birth_date,wrestler_2_height,wrestler_2_weight,wrestler_1_age,wrestler_2_age
115476,2004-01-01,2,149,J14w,Juzan,1-1 (9-6),0,oshidashi,1123,J12e,Hakuho,2-0 (9-6),1,Oshiogawa,Ibaraki,1975-04-08,180.5,165.0,Miyagino,Mongolia,1985-03-11,187.0,101.5,29,19
115477,2004-01-01,8,15,J13w,Oginishiki,4-4 (4-9),0,kotenage,1123,J12e,Hakuho,6-2 (9-6),1,Dewanoumi,Chiba,1971-07-08,186.0,135.5,Miyagino,Mongolia,1985-03-11,187.0,101.5,33,19
115478,2004-01-01,5,222,J14e,Sumanofuji,5-0 (8-7),1,oshidashi,1123,J12e,Hakuho,3-2 (9-6),0,Nakamura,Hyogo,1977-12-21,176.0,150.0,Miyagino,Mongolia,1985-03-11,187.0,101.5,27,19
115479,2004-01-01,10,103,J12w,Tochifudo,2-8 (5-10),0,yorikiri,1123,J12e,Hakuho,7-3 (9-6),1,Kasugano,Saitama,1977-06-14,190.5,160.5,Miyagino,Mongolia,1985-03-11,187.0,101.5,27,19
115480,2004-01-01,9,874,J13e,Hamanishiki,6-3 (8-7),1,yorikiri,1123,J12e,Hakuho,6-3 (9-6),0,Oitekaze,Kumamoto,1976-11-23,181.5,129.5,Miyagino,Mongolia,1985-03-11,187.0,101.5,28,19


In [11]:
#Save file of desired data for project use 
sumo_df.to_csv('sumo_data.csv', index = False)
files.download('sumo_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>